In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import FewShotPromptTemplate, FewShotChatMessagePromptTemplate

llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template(
    "What is the capital of {country}? Please provide up to {n_words} words of explanation.",
)

# Passando direto:
prompt2 = PromptTemplate.from_template(
    "What is the capital of {country}? Please provide up to {n_words} words of explanation.", partial_variables={"country": "France", "n_words": 5}
)

# prompt = prompt.format(country="France", n_words=5)
# print(prompt)
# llm.invoke(prompt)


# Utilizando múltiplos prompts 

template_word = PromptTemplate.from_template(
    "What is the capital of {country}?",
)

template_line_count = PromptTemplate.from_template(
    "Please provide up to {n_lines} lines of explanation.",
)

template_language = PromptTemplate.from_template(
    "Please provide the answer in {language}.",
)

final_prompt = (template_word + template_line_count + template_language + " Responda seguindo a {pergunta}") 

# print(final_prompt)
prompt = final_prompt.format(
    country="Mexico",
    n_lines=5,
    language="Portuguese",
    pergunta="pergunta acima"
)

# llm.invoke(prompt)

# Templates para Chat
# chat_template = ChatPromptTemplate.from_template("Essa é a minha pergunta: {question}")
# chat_template.format_messages(
#     question="Qual é a capital do Brasil?")

# System é o papel
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente que responde perguntas de modo irônico e zueiro. Me chamo {nome}"),
    ("human", "Olá, como você está?"),
    ("ai", "Estou bem, como posso te ajudar?"),
    ("human", "{pergunta}"),
])

# print(chat_template)

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
# response = chat.invoke(
#     chat_template.format_messages(
#         nome="João",
#         pergunta="Qual é a capital do Brasil?"
#     )
# )


# print(response.content)

# Few Shot Prompting
exemplos = [
    {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 
""", 
    }, 
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 
""", 
    }, 
    {"pergunta": "Quem foi o pai de Napoleão Bonaparte?",
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte? 
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês. 
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte? 
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Buonaparte. 
Então a resposta final é: Carlo Buonaparte 
""", 
    },
    {"pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'? 
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson. 
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'? 
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson. 
Então a resposta final é: Sim 
""",
    },
]

example_prompt = PromptTemplate(
    input_variables=["pergunta", "resposta"],
    template="Pergunta {pergunta}\n{resposta}"
)

example_prompt.format(**exemplos[0]) 

prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}", 
    input_variables=["input"], 
)

# print(prompt.format(input="Quem é melhor, Messi ou Cristiano Ronaldo?"))
# llm.invoke(prompt.format(input="Quem é melhor, Messi ou Cristiano Ronaldo? Você não pode ficar em cima do muro! Com base em dados estatísticos, gols, vitórias e títulos, quem é o melhor jogador de futebol?"))

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{pergunta}"),
     ("ai", "{resposta}")]
)

example_prompt.format_messages(**exemplos[0])

few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_final = ChatPromptTemplate.from_messages([
    few_shot_template,
    ("human", "{input}")
])

prompt = prompt_final.format_messages(input="Quem fez mais gols, Messi ou Cristiano Ronaldo?")
response = chat.invoke(prompt)
print(response.content)